## Import Necessary libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#Data Cleaning and Preprocessing Libraries
from parse_10k.parse_10K import parse_10k_filing


#Fine-Tuning Libraries
from datasets import load_dataset
import json
import subprocess
from mlx_lm import load

#RAG Related Libraries
import sys
import os
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

/opt/homebrew/anaconda3/envs/Fintech/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download the Llama Model

In [2]:
hf_model_path = "meta-llama/Meta-Llama-3-8B-Instruct"


## Quantize the Model

In [3]:
def run_command_with_live_output(command: list[str]) -> None:
    """
    Runs a command and prints its output line by line as it executes.

    Args:
        command (List[str]): The command and its arguments to be executed.

    Returns:
        None
    """
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Print the output line by line
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
        
    # Print the error output, if any
    err_output = process.stderr.read()
    if err_output:
        print(err_output)

In [4]:

command = ['python', '-m', 'mlx_lm.convert', '--hf-path', hf_model_path, '-q']

python -m mlx_lm.convert --hf-path meta-llama/Meta-Llama-3-8B-Instruct -q


In [ ]:
run_command_with_live_output(command)

## Prepare Data for Fine-Tuning

In [5]:
model, tokenizer = load(path_or_hf_repo='mlx_model')

In [6]:
folder = 'data/'
system_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}

### Context:
{}

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""

def format_prompt(examples):
    questions = examples["question"]
    contexts = examples["context"]
    responses = examples["answer"]
    texts = []
    for question, context, response in zip(questions, contexts, responses):
        text = system_prompt.format(question, context, response)
        texts.append(text)
    return { "text" : texts, }

dataset = load_dataset("virattt/llama-3-8b-financialQA", split="train")
dataset = dataset.shuffle().select(range(150))
dataset = dataset.map(format_prompt, batched=True)
dataset = dataset.train_test_split(test_size=50/150)
dataset_test_valid = dataset['test'].train_test_split(0.5)

print(dataset["train"][45]["text"])  # Modified to print only the 'text' field

# Save the datasets to .jsonl files
def save_jsonl(data, filename):
    with open(folder + filename, "w") as f:
        for entry in data:
            f.write(json.dumps({"text": entry["text"]}) + "\n")

save_jsonl(dataset["train"], "train.jsonl")
save_jsonl(dataset_test_valid["train"], "test.jsonl")
save_jsonl(dataset_test_valid["test"], "valid.jsonl")

Map: 100%|██████████| 150/150 [00:00<00:00, 12343.45 examples/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
By what amount did fiscal 2022 diluted earnings per share increase from fiscal 2021?

### Context:
Diluted earnings per share were $16.69 in fiscal 2022 compared to $15.53 in fiscal 2021.

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
$1.16


In [ ]:
config_path = 'lora_config.yaml'
train_command = ['mlx_lm.lora', '--config', config_path]

run_command_with_live_output(train_command)

## Run Inference on the Fine-Tuned Model

In [7]:
model_path = 'mlx_model'
adapters_path = 'adapters'
max_tokes = str(2048)
EOS_TOKEN = tokenizer.eos_token

context = "The increase in research and development expense for fiscal year 2023 was primarily driven by increased compensation, employee growth, engineering development costs, and data center infrastructure."
question = "What were the primary drivers of the notable increase in research and development expenses for fiscal year 2023?"

prompt = system_prompt.format(question, context, "")

inf_command = ['mlx_lm.generate', '--model', model_path, '--adapter-path', adapters_path, '--eos-token', EOS_TOKEN, '--max-tokens', max_tokes, '--prompt', prompt]

run_command_with_live_output(inf_command)

Prompt: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
What were the primary drivers of the notable increase in research and development expenses for fiscal year 2023?

### Context:
The increase in research and development expense for fiscal year 2023 was primarily driven by increased compensation, employee growth, engineering development costs, and data center infrastructure.

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|><|eot_id|><|start_header_id|>assistant<|end_header_id|>


The primary drivers of the notable increase in research and development expenses for fiscal year 2023 were increased compensation, employee growth, engineering development costs, and data cent

## Part-2 Using langchain for RAG

In [9]:
file_path = '10-k_data/sec-edgar-filings/AAPL/10-K/AAPL_18/AAPL_18.html'
section = 3

filing_text = parse_10k_filing(file_path, section)
filing_text

["Item 7.Managements Discussion and Analysis of Financial Condition and Results of OperationsThis section and other parts of this Annual Report on Form 10-K (Form 10-K) contain forward-looking statements, within the meaning of the Private Securities Litigation Reform Act of 1995, that involve risks and uncertainties. Forward-looking statements provide current expectations of future events based on certain assumptions and include any statement that does not directly relate to any historical or current fact. Forward-looking statements can also be identified by words such as future,  anticipates,  believes,  estimates,  expects,  intends,  plans,  predicts,  will,  would,  could,  can,  may,  and similar terms. Forward-looking statements are not guarantees of future performance and the Companys actual results may differ significantly from the results discussed in the forward-looking statements. Factors that might cause such differences include, but are not limited to, those discussed in P

In [10]:
# HF Model Path
modelPath = "BAAI/bge-large-en-v1.5"

model_kwargs = {'device':'mps'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of LangChain's HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     
    model_kwargs=model_kwargs, 
    encode_kwargs=encode_kwargs,
)

/opt/homebrew/anaconda3/envs/Fintech/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5


In [11]:
# Initialize a text splitter to divide the filing data into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,         
    chunk_overlap = 500,       
    length_function = len,     
    is_separator_regex = False 
)
# Split the filing data into smaller, manageable chunks
split_data = text_splitter.create_documents(filing_text)

# Create a FAISS vector database from the split data using embeddings
db = FAISS.from_documents(split_data, embeddings)

# Create a retriever object to search within the vector database
retriever = db.as_retriever()


INFO:faiss.loader:Loading faiss.
Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
Successfully loaded faiss.


In [12]:
# Retrieval Function
def retrieve_context(query):
    global retriever
    retrieved_docs = retriever.invoke(query) 
    context = []
    for doc in retrieved_docs:
        context.append(doc.page_content) 
    return context

context = retrieve_context("How have currency fluctuations impacted the company's net sales and gross margins?")
print(context)

['the U.S. dollar. Accordingly, changes in exchange rates, and in particular a strengthening of the U.S. dollar, will negatively affect the Companys net sales and gross margins as expressed in U.S. dollars. There is a risk that the Company will have to adjust local currency product pricing due to competitive pressures when there has been significant volatility in foreign currency exchange rates.The Company may enter into foreign currency forward and option contracts with financial institutions to protect against foreign exchange risks associated with certain existing assets and liabilities, certain firmly committed transactions, forecasted future cash flows and net investments in foreign subsidiaries. In addition, the Company has entered, and in the future may enter, into foreign currency contracts to partially offset the foreign currency exchange gains and losses on its foreign currencydenominated debt issuances. The Company generally hedges portions of its forecasted foreign currency

In [13]:

def run_command(command: list[str]) -> str:

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Capture stdout and stderr
    stdout, stderr = process.communicate()

    # Combine stdout and stderr
    full_output = stdout + stderr

    return full_output.strip()

def inference(question, context):
    model_path = 'mlx_model'
    adapters_path = 'adapters'
    max_tokes = str(2048)
    EOS_TOKEN = tokenizer.eos_token

    prompt = system_prompt.format(question, context, "")

    inf_command = ['mlx_lm.generate', '--model', model_path, '--adapter-path', adapters_path, '--eos-token', EOS_TOKEN, '--max-tokens', max_tokes, '--prompt', prompt]
    
    output = run_command(inf_command)
    
    return output
def extract_response(text):
    # Remove the assumption that text is a list
    if isinstance(text, list):
        text = text[0]
    
    start_marker = "### Response:"
    end_marker = "=========="

    start_index = text.find(start_marker)
    if start_index == -1:
        return None
    start_index += len(start_marker)

    end_index = text.find(end_marker, start_index)
    if end_index == -1:
        end_index = len(text)

    response = text[start_index:end_index].strip()

    # Remove any remaining header tags
    response = response.replace("<|start_header_id|>assistant<|end_header_id|>", "")
    response = response.replace("<|eot_id|>", "")

    return response.strip()

In [14]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
while True:
    question = input(f"What would you like to know about AAPL's form 10-K? ")
    if question == "x":
        break
    else:
        context = retrieve_context(question) 
        print(f"User: {question}")
        resp = inference(question, context) # Running Inference
        parsed_response = extract_response(resp) 
        print(f"L3 Agent: {parsed_response}")
        print("-----\n")

User: What is the net income of AAPL in 2018?
L3 Agent: The net income of Apple Inc. in 2018 was $59.5 billion.
-----

User: What are the major revenue streams for AAPL?
L3 Agent: AAPL's major revenue streams include net sales from iPhone, Services and Other Products, primarily generated by the Americas segment, which accounted for approximately 42% of total net sales in 2018.
-----

User: What is the net sales by product for AAPL in 2018?
L3 Agent: The net sales by product for AAPL (Apple Inc.) in 2018 were:

* iPhone: $166,699 million
* iPad: $18,805 million
* Mac: $25,484 million
* Services: $37,190 million
* Other Products: $17,417 million
-----

User: What is the sales of each revenue streams in $?
L3 Agent: The sales of each revenue stream in $ are:

1. Net sales by reportable segment:
	* Americas: $112,093 million (16% increase)
	* Europe: $62,420 million (14% increase)
	* Greater China: $51,942 million (16% increase)
	* Japan: $21,733 million (23% increase)
	* Rest of Asia Paci